In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyproj
import folium
import datetime as DT
%matplotlib inline

In [20]:
df = pd.read_csv(os.getcwd() + '/Data/parking-citations.csv', low_memory=False)

In [21]:
#updating formatting so that I can translate issue date to datetime
df['Issue Date'] = df[df['Issue Date'].notnull()]['Issue Date'].apply(lambda x: x.split('T')[0])
df['Issue Date'] = pd.to_datetime(df['Issue Date'], infer_datetime_format=True)

#limiting dataset so it's easy to work with
today = pd.Timestamp('today').normalize()
week_ago = today - DT.timedelta(days=7)
month_ago = today - DT.timedelta(days=30)
df = df[df['Issue Date']>=month_ago]

In [22]:
#pad anything that is less than 4 digits then isolate just the hours
df['Issue time'] = df['Issue time'].astype(str)
df['Issue time'] = df['Issue time'].apply(lambda x: x.split('.')[0])
df['Issue time'] = df[df['Issue time'].notnull()]['Issue time'].apply(lambda x: x.zfill(4))
df['Issue Hour'] = df[df['Issue time']!='0nan']['Issue time'].apply(lambda x: DT.datetime.strptime(x,'%H%M').hour)

#clean lat lon
df['Latitude'] = np.where(df['Latitude']==99999.000, np.nan, df['Latitude'])
df['Longitude'] = np.where(df['Longitude']==99999.000, np.nan, df['Longitude'])

#string for ticketnum
df['Ticket number'] = df['Ticket number'].astype(str)

In [23]:
import pyproj
pm = '+proj=lcc +lat_1=34.03333333333333 +lat_2=35.46666666666667 +lat_0=33.5 +lon_0=-118 +x_0=2000000 +y_0=500000.0000000002 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'
x1m,y1m = df['Latitude'].values, df['Longitude'].values
x2m,y2m = pyproj.transform(pyproj.Proj(pm,preserve_units = True), pyproj.Proj("+init=epsg:4326"), x1m,y1m)
df['Latitude']=x2m
df['Longitude']=y2m

In [24]:
import folium

LA_COORDINATES = (34.05, -118.24)

# create empty map zoomed in on San Francisco
map = folium.Map(location=LA_COORDINATES, zoom_start=10) 

# add a marker for every record in the filtered data, use a clustered view
from folium.plugins import FastMarkerCluster
FastMarkerCluster(data=list(zip(df[(df['Issue Date']>week_ago) & (df['Longitude'].notnull())]['Longitude'],(df[(df['Issue Date']>week_ago) & (df['Latitude'].notnull())]['Latitude'])))).add_to(map)

folium.LayerControl().add_to(map)
    
display(map)

In [27]:
df[df['Longitude'].notnull()].to_csv(os.getcwd() + '/Data/parking-citations_Geo.csv')